In [ ]:
import datetime as dt
print(f"This lesson was developed on: {dt.date.today()}")

**Table of contents**<a id='toc0_'></a>    
- [API Wrappers](#toc1_)    
- [Twitter API: RIP ⚰️](#toc2_)    
- [Google Sheets API: A new beginning](#toc3_)    
  - [Setup](#toc3_1_)    
  - [Accessing a spreadsheet (look, mom, no credentials!)](#toc3_2_)    
  - [Handling credentials in mini-production](#toc3_3_)    
  - [`gspread` API wrapper, here to make life easier](#toc3_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[API Wrappers](#toc0_)

The job of an API wrapper is to simplify even further the access to the data of a website. An API wrapper is typically a (Python) library that allows access to different API endpoints in an object-oriented manner. 

# <a id='toc2_'></a>[Twitter API: RIP ⚰️](#toc0_)

![Elon-murderer](https://media.giphy.com/media/JVCY1XbJtVtdREJFWO/giphy.gif)

As much as I wanted to share with you how the Twitter API wrapper (`tweepy`) works, Elon has put so many restrictions on the free version that the only available options are to manage tweets and to look up users...

![Alt text](../images/twitter_api.png)

...unless you're keen to pay at least $100/month for the Basic version, of course.

# <a id='toc3_'></a>[Google Sheets API: A new beginning](#toc0_)

![excel](https://media.giphy.com/media/UMyvk17PIo3SiZQWju/giphy.gif)

Today we will be looking at an API wrapper for working with Google Sheets, which can come in handy if you need to automate tasks or connect Google Sheets to different web applications. 

## <a id='toc3_1_'></a>[Setup](#toc0_)

First we'll install the [`gspread`](https://docs.gspread.org/en/latest/) library:

In [ ]:
# You know the drill
# !pip install --upgrade gspread

Now we will create a service account:

1. Go to https://console.developers.google.com/
2. Click "Create project" (top right)
3. Click "Enable APIs and Services" (next to project title)
4. Enable the Google Sheets API for your project (select the Google Sheets option), then click "Enable"
5. Go to Create Credentials > Service Account > Name Service Account > Create & Continue > Role = Owner > Share with other people > Done
6. Select Credentials > Service Accounts > Select the service you just created > Keys (top menu) > ADD KEY > Create new key > JSON > Download in the same folder as this notebook

**Note:** The reason we create a service account is to provide continuity. For example, if someone was using their work account but then left the company, their account would be deleted and the company access to the API would be lost. Therefore, a service account is there to prevent that from happening.

## <a id='toc3_2_'></a>[Accessing a spreadsheet (look, mom, no credentials!)](#toc0_)

In [ ]:
import requests
import gspread
import json
import pandas as pd

For this exercise, I recommend you create a copy of the [[DEMO] Course Planning](https://docs.google.com/spreadsheets/d/1lOOM9GrIiRIOHVzWjOyu12mVC-5PbLnTRpeIzrrLywI/edit#gid=955120891) spreadsheet so we don't interfere in the changes we make to the sheet.

In [ ]:
# Set up os and dotenv
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Get spreadsheet ID
spreadsheet_id = os.getenv('SPREADSHEET_ID')
print(spreadsheet_id)

In [ ]:
# Select a range and create a new URL
range_name = "'Class Planning'!A1"
url = 'https://sheets.googleapis.com/v4/spreadsheets/' + spreadsheet_id + '/values/' + range_name

**Note:** The spreadsheet ID doesn't change once the sheet has been created so you can access it even if it was moved to a different location in GDrive!

In [ ]:
# Try to extract the contents of the cell
response = requests.get(url)
response.json()

## <a id='toc3_3_'></a>[Handling credentials in mini-production](#toc0_)

Google Developer setup (part 2):

- Create a new API key in the **Credentials > Create Credentials > API key** and save it to your `.env` file
- Share the [[DEMO] Course Schedule](https://docs.google.com/spreadsheets/d/1lOOM9GrIiRIOHVzWjOyu12mVC-5PbLnTRpeIzrrLywI/edit#gid=955120891) sheet with your Service email (looks something like this: `service-name@service-name.iam.gserviceaccount.com`)

⚠️ **Again, make sure to add the `.env` file in your `.gitignore` file if you're uploading anything on GitHub!!!** ⚠️

In [ ]:
# Get API key
API_key = os.getenv('GOOGLE_API_KEY')
print(API_key)

In [ ]:
# Set up request parameters & headers
params = {
   'key': API_key
}

In [ ]:
# Send request
response = requests.get(url, params=params)

In [ ]:
# Review response
response.json()

> This is **not** how we would typically handle a request in production, for that we would use an more advanced authentication method, such as OAuth, which essentially generates Authentication keys with a timeout. However that is kind of a bother to handle, requiring even a call to *another* API to generate the keys.

## <a id='toc3_4_'></a>[`gspread` API wrapper, here to make life easier](#toc0_)

In [ ]:
json_credentials_path = 'class_credentials.json'

In [ ]:
# Connect to the service account
gc = gspread.service_account(filename=json_credentials_path)

In [ ]:
# Get first workseet
sheet = gc.open_by_key(spreadsheet_id)
worksheet = sheet.get_worksheet(0)

In [ ]:
# Show worksheet
worksheet

In [ ]:
# or use sheet name
worksheet = sheet.worksheet("Class Planning")

Get all values is not available in the original API (where you need to specify the range), but it's a natural call to make:

In [ ]:
# Get all values
worksheet.get_all_values()

or in a different form, which may be more helpful, but would not be available natively:

In [ ]:
# get all records
print(worksheet.get_all_records())

# Show them into a dataframe
pd.DataFrame(worksheet.get_all_records())

**Note:** If any of your Google Sheets headers are duplicates, you will get an error (`GSpreadException: the given 'expected_headers' are not uniques`)

In [ ]:
# Find all the cells containing the projects using regex
import re
worksheet.findall(re.compile('project'))

As another example, the native API has functions to add a row at once, which would require a loop and multiple requests if we wanted to add a bunch of rows, but this is a natural action to do with the API:

In [ ]:
# Define the index and number of rows to insert
row_index = len(pd.DataFrame(worksheet.get_all_records())) + 1
num_rows = 4

# Perform batch insertion
worksheet.insert_rows(range(num_rows), row=row_index)

To learn more about this API Wrapper, you can visit the [documentation](https://developers.google.com/sheets/api/guides/concepts). Services that do integrations between multiple apps (Zapier, Airtable) very likely use the Google Sheets API to communicate across services.